# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:44<00:00, 32.92s/it]


In [4]:
len(deals)

50

In [16]:
deals[0].describe()

"Title: Shokz OpenRun Pro 2 Bone Conduction Sport Headphones for $120 + free shipping\nDetails: It's the lowest price we could find by $60 and the best deal we've seen for this model. Buy Now at eBay\nFeatures: \nURL: https://www.dealnews.com/products/Shokz/Shokz-Open-Run-Pro-2-Bone-Conduction-Sport-Headphones/487107.html?iref=rss-c142"

In [17]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [18]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [19]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Shokz OpenRun Pro 2 Bone Conduction Sport Headphones for $120 + free shipping
Details: It's the lowest price we could find by $60 and the best deal we've seen for this model. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/products/Shokz/Shokz-Open-Run-Pro-2-Bone-Conduction-Sport-Headphones/487107.html?iref=rss-c142

Title: Open-Box Clearance Deals at Neweg

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The Panasonic Lumix S PRO 16-35mm f/4 lens is a versatile and high-performance lens designed for L Mount cameras. This lens features a constant f/4 aperture, allowing photographers to achieve stunning clarity and detail in various lighting conditions. It comes bundled with essential accessories like a ProOptic 77mm filter kit, a lens cleaner, a lens wrap cap, and software, providing a comprehensive photography package. Its weather-sealed construction ensures durability during outdoor shoots, making it an ideal choice for both professional and amateur photographers seeking high quality imagery.', price=998.0, url='https://www.dealnews.com/Panasonic-Lumix-S-PRO-16-35-mm-f-4-Lens-for-L-Mount-Bundle-for-998-free-shipping/21738418.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description="The Shokz OpenRun Pro 2 Bone Conduction Sport Headphones provide a unique listening experience that allows you to hear your surroundings while enjoying music or calls. These headphones utilize bone conduction technology, delivering sound through your cheekbones instead of in your ears, ensuring comfort and safety during outdoor activities. Lightweight and sweatproof, they're ideal for workouts or casual use, with a battery life that keeps up with your active lifestyle.", price=120.0, url='https://www.dealnews.com/products/Shokz/Shokz-Open-Run-Pro-2-Bone-Conduction-Sport-Headphones/487107.html?iref=rss-c142'), Deal(product_description='The Panasonic Lumix S PRO 16-35mm f/4 Lens is designed for L Mount cameras, featuring a versatile focal length ideal for a range of photography styles from landscapes to architecture. With professional optical performance, this lens includes advanced elements to minimize distortion and enhance image quality. 